# 5.3 Leave-One-Study-Out Evaluation (Value + Condition)

**模型架构**: Value + Condition (Full vCross, No Prior)
**任务**: 对 6 个目标 Study 进行留一法交叉验证 (LOO-CV) 和零样本迁移 (Zero-shot) 测试。

In [1]:
import os
import torch
import numpy as np
import pandas as pd
import json
import joblib
import gc
import shutil
from argparse import Namespace
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

from torch.utils.data import Subset
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

# --- v2.0 自定义模块 ---
from MiCoGPT.utils_vCross.model_vCross import MiCoGPTConfig, MiCoGPTForSequenceClassification
from MiCoGPT.utils_vCross.collator_vCross import MiCoGPTClassificationCollator

# --- 复用工具函数 ---
from MiCoGPT.utils.finetune_v2 import SubsetWithLabels
from MiCoGPT.utils.mgm_utils import eval_and_save

import warnings
warnings.filterwarnings("ignore")

In [2]:
args = Namespace(
    # 语料库 (vCross 标准版)
    input="../data/vCross/ResMicroDB_90338_vCross.pkl",
    
    # 基础模型 (ValueCondition Group A)
    base_model="../models/finetuned_vCross_value_condition_valLoss",
    
    # 标签编码器
    label_encoder_path="../models/finetuned_vCross_value_condition_valLoss/label_encoder.joblib",
    
    # 输出根目录
    output_root="../models/LOO_vCross_value_condition_valLoss",
    
    # 目标 Studies
    target_studies = [
        'PRJNA439311', 'PRJNA282010', 'PRJNA296567', 
        'PRJNA632472', 'PRJNA1108737', 'PRJNA820972'
    ],
    
    label_col="Is_Healthy",
)

os.makedirs(args.output_root, exist_ok=True)

In [3]:
print("Loading Corpus...")
with open(args.input, "rb") as f:
    import pickle
    corpus = pickle.load(f)

print("Loading Label Encoder...")
le = joblib.load(args.label_encoder_path)

# 兼容 LabelEncoder 和 OneHotEncoder
if hasattr(le, "classes_"):
    classes = le.classes_
else:
    classes = le.categories_[0]
print(f"Classes: {classes}")

# 筛选 Split_Group == 'C' 且在目标 Studies 中的样本
meta = corpus.metadata
mask = (
    (meta["Split_Group"] == "C") & 
    (meta["Project_ID"].isin(args.target_studies)) & 
    (meta[args.label_col].notna())
)
target_indices = np.where(mask)[0]
target_subset = Subset(corpus, target_indices)
print(f"Selected {len(target_subset)} samples from {len(args.target_studies)} studies.")

Loading Corpus...
Loading Label Encoder...
Classes: [False True]
Selected 1029 samples from 6 studies.


In [4]:
# 定义标准 Collator (不屏蔽任何输入)
collator = MiCoGPTClassificationCollator(corpus.tokenizer, max_length=512)

In [5]:
def get_study_subset(study_id, indices_map):
    study_mask = (meta["Project_ID"] == study_id) & mask
    study_indices = np.where(study_mask)[0]
    return Subset(corpus, study_indices)

def prepare_dataset(subset, le):
    indices = subset.indices
    labels_raw = corpus.metadata.iloc[indices][args.label_col].values

    if hasattr(le, "classes_"):
        labels_encoded = le.transform(labels_raw)
    else:
        # OneHotEncoder
        labels_onehot = le.transform(labels_raw.reshape(-1, 1))
        if hasattr(labels_onehot, "toarray"):
            labels_onehot = labels_onehot.toarray()
        labels_encoded = np.argmax(labels_onehot, axis=1)

    return SubsetWithLabels(subset, torch.tensor(labels_encoded, dtype=torch.long))

## 任务一：Zero-shot Evaluation

In [6]:
print("=== Starting Zero-shot Evaluation ===")

model = MiCoGPTForSequenceClassification.from_pretrained(
    args.base_model, 
    num_labels=len(classes),
    ignore_mismatched_sizes=True
)

trainer = Trainer(
    model=model,
    data_collator=collator,
    args=TrainingArguments(output_dir=".tmp", per_device_eval_batch_size=32, eval_accumulation_steps=None)
)

for study in args.target_studies:
    print(f"Evaluating on {study}...")
    study_subset = get_study_subset(study, None)
    if len(study_subset) == 0: continue
    test_ds = prepare_dataset(study_subset, le)
    
    predictions = trainer.predict(test_ds)
    y_score = predictions.predictions
    y_true = predictions.label_ids
    
    save_dir = f"{args.output_root}/zero_shot/{study}"
    os.makedirs(save_dir, exist_ok=True)
    eval_and_save(y_score, y_true, [str(c) for c in classes], save_dir, activation="softmax")
    print(f"Saved to {save_dir}")

del model, trainer
gc.collect()
torch.cuda.empty_cache()

=== Starting Zero-shot Evaluation ===
Evaluating on PRJNA439311...


Evaluating biome source: False
       TN  FP  FN  TP     Acc      Sn      Sp     TPR     FPR      Rc      Pr  \
t                                                                               
0.000   0  16   0  86  0.8431  1.0000  0.0000  1.0000  1.0000  1.0000  0.8431   
0.001   0  16   0  86  0.8431  1.0000  0.0000  1.0000  1.0000  1.0000  0.8431   
0.002   0  16   0  86  0.8431  1.0000  0.0000  1.0000  1.0000  1.0000  0.8431   
0.003   0  16   0  86  0.8431  1.0000  0.0000  1.0000  1.0000  1.0000  0.8431   
0.004   0  16   0  86  0.8431  1.0000  0.0000  1.0000  1.0000  1.0000  0.8431   
...    ..  ..  ..  ..     ...     ...     ...     ...     ...     ...     ...   
0.997   6  10   6  80  0.8431  0.9302  0.3750  0.9302  0.6250  0.9302  0.8889   
0.998   9   7  13  73  0.8039  0.8488  0.5625  0.8488  0.4375  0.8488  0.9125   
0.999  11   5  30  56  0.6569  0.6512  0.6875  0.6512  0.3125  0.6512  0.9180   
1.000  16   0  86   0  0.1569  0.0000  1.0000  0.0000  0.0000  0.0000  0.0000 

Evaluating biome source: False
       TN  FP  FN  TP     Acc      Sn      Sp     TPR     FPR      Rc      Pr  \
t                                                                               
0.000   0  19   0  18  0.4865  1.0000  0.0000  1.0000  1.0000  1.0000  0.4865   
0.001   0  19   0  18  0.4865  1.0000  0.0000  1.0000  1.0000  1.0000  0.4865   
0.002   0  19   0  18  0.4865  1.0000  0.0000  1.0000  1.0000  1.0000  0.4865   
0.003   0  19   0  18  0.4865  1.0000  0.0000  1.0000  1.0000  1.0000  0.4865   
0.004   0  19   0  18  0.4865  1.0000  0.0000  1.0000  1.0000  1.0000  0.4865   
...    ..  ..  ..  ..     ...     ...     ...     ...     ...     ...     ...   
0.997   5  14   1  17  0.5946  0.9444  0.2632  0.9444  0.7368  0.9444  0.5484   
0.998   8  11   2  16  0.6486  0.8889  0.4211  0.8889  0.5789  0.8889  0.5926   
0.999  11   8   6  12  0.6216  0.6667  0.5789  0.6667  0.4211  0.6667  0.6000   
1.000  19   0  18   0  0.5135  0.0000  1.0000  0.0000  0.0000  0.0000  0.0000 

Evaluating biome source: False
       TN  FP  FN  TP     Acc    Sn   Sp   TPR  FPR    Rc      Pr      F1  \
t                                                                           
0.000   0  14   0  20  0.5882  1.00  0.0  1.00  1.0  1.00  0.5882  0.7407   
0.001   0  14   0  20  0.5882  1.00  0.0  1.00  1.0  1.00  0.5882  0.7407   
0.002   0  14   0  20  0.5882  1.00  0.0  1.00  1.0  1.00  0.5882  0.7407   
0.003   0  14   0  20  0.5882  1.00  0.0  1.00  1.0  1.00  0.5882  0.7407   
0.004   0  14   0  20  0.5882  1.00  0.0  1.00  1.0  1.00  0.5882  0.7407   
...    ..  ..  ..  ..     ...   ...  ...   ...  ...   ...     ...     ...   
0.997  14   0  19   1  0.4412  0.05  1.0  0.05  0.0  0.05  1.0000  0.0952   
0.998  14   0  19   1  0.4412  0.05  1.0  0.05  0.0  0.05  1.0000  0.0952   
0.999  14   0  20   0  0.4118  0.00  1.0  0.00  0.0  0.00  0.0000     NaN   
1.000  14   0  20   0  0.4118  0.00  1.0  0.00  0.0  0.00  0.0000     NaN   
1.001  14   0  20   0  0.4118  0.00  1.0  0.0

Evaluating biome source: False
        TN   FP   FN   TP     Acc      Sn      Sp     TPR     FPR      Rc  \
t                                                                           
0.000    0  207    0  338  0.6202  1.0000  0.0000  1.0000  1.0000  1.0000   
0.001    0  207    0  338  0.6202  1.0000  0.0000  1.0000  1.0000  1.0000   
0.002    0  207    0  338  0.6202  1.0000  0.0000  1.0000  1.0000  1.0000   
0.003    0  207    0  338  0.6202  1.0000  0.0000  1.0000  1.0000  1.0000   
0.004    0  207    0  338  0.6202  1.0000  0.0000  1.0000  1.0000  1.0000   
...    ...  ...  ...  ...     ...     ...     ...     ...     ...     ...   
0.997   51  156   47  291  0.6275  0.8609  0.2464  0.8609  0.7536  0.8609   
0.998   71  136   68  270  0.6257  0.7988  0.3430  0.7988  0.6570  0.7988   
0.999  110   97  118  220  0.6055  0.6509  0.5314  0.6509  0.4686  0.6509   
1.000  207    0  338    0  0.3798  0.0000  1.0000  0.0000  0.0000  0.0000   
1.001  207    0  338    0  0.3798  0.0000  1.

Evaluating biome source: False
       TN  FP  FN  TP     Acc      Sn      Sp     TPR     FPR      Rc      Pr  \
t                                                                               
0.000   0  76   0  76  0.5000  1.0000  0.0000  1.0000  1.0000  1.0000  0.5000   
0.001   0  76   0  76  0.5000  1.0000  0.0000  1.0000  1.0000  1.0000  0.5000   
0.002   0  76   1  75  0.4934  0.9868  0.0000  0.9868  1.0000  0.9868  0.4967   
0.003   1  75   1  75  0.5000  0.9868  0.0132  0.9868  0.9868  0.9868  0.5000   
0.004   1  75   1  75  0.5000  0.9868  0.0132  0.9868  0.9868  0.9868  0.5000   
...    ..  ..  ..  ..     ...     ...     ...     ...     ...     ...     ...   
0.997  70   6  65  11  0.5329  0.1447  0.9211  0.1447  0.0789  0.1447  0.6471   
0.998  70   6  65  11  0.5329  0.1447  0.9211  0.1447  0.0789  0.1447  0.6471   
0.999  74   2  69   7  0.5329  0.0921  0.9737  0.0921  0.0263  0.0921  0.7778   
1.000  76   0  76   0  0.5000  0.0000  1.0000  0.0000  0.0000  0.0000  0.0000 

Evaluating biome source: False
       TN  FP   FN   TP     Acc      Sn      Sp     TPR     FPR      Rc  \
t                                                                         
0.000   0  54    0  105  0.6604  1.0000  0.0000  1.0000  1.0000  1.0000   
0.001   0  54    0  105  0.6604  1.0000  0.0000  1.0000  1.0000  1.0000   
0.002   0  54    0  105  0.6604  1.0000  0.0000  1.0000  1.0000  1.0000   
0.003   0  54    0  105  0.6604  1.0000  0.0000  1.0000  1.0000  1.0000   
0.004   0  54    0  105  0.6604  1.0000  0.0000  1.0000  1.0000  1.0000   
...    ..  ..  ...  ...     ...     ...     ...     ...     ...     ...   
0.997  36  18   51   54  0.5660  0.5143  0.6667  0.5143  0.3333  0.5143   
0.998  40  14   60   45  0.5346  0.4286  0.7407  0.4286  0.2593  0.4286   
0.999  46   8   74   31  0.4843  0.2952  0.8519  0.2952  0.1481  0.2952   
1.000  54   0  105    0  0.3396  0.0000  1.0000  0.0000  0.0000  0.0000   
1.001  54   0  105    0  0.3396  0.0000  1.0000  0.0000  0.0000  0.00

## 任务二：LOO Fine-tuning

In [7]:
import numpy as np
import gc
import torch
from transformers import TrainerCallback

print("=== Starting Nested LOO (Double LOO) Fine-tuning ===")

# 辅助函数：清理显存
def clean_memory():
    gc.collect()
    torch.cuda.empty_cache()

# 辅助函数：计算指标
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {"accuracy": acc, "f1": f1}

# 定义最大搜索轮数
MAX_EPOCHS = 50

# === 1. 外层循环：留一个作为最终测试集 (Test Study) ===
for test_study in args.target_studies:
    print(f"\n{'='*20}")
    print(f">>> Outer Loop: Test Target = {test_study}")
    print(f"{'='*20}")
    
    # 确定当前的训练池 (5个 studies)
    train_pool_studies = [s for s in args.target_studies if s != test_study]
    
    # 用于存储内层 5 折的 Loss 曲线
    # 结构: [ [fold1_epoch1_loss, fold1_epoch2_loss...], [fold2...], ... ]
    all_folds_val_losses = []
    
    # === 2. 内层循环：5-Fold CV 寻找最佳 Epoch ===
    print(f"\n[Inner Loop] Starting 5-fold CV on train pool to find optimal epochs...")
    
    for fold_idx, val_study in enumerate(train_pool_studies):
        print(f"  Inner Fold {fold_idx+1}/5: Val Study = {val_study}")
        
        # 2.1 准备内层数据
        # 内层训练集：训练池中除了当前验证 Study 之外的 4 个
        inner_train_studies = [s for s in train_pool_studies if s != val_study]
        
        # 构建 Dataset
        inner_train_mask = (meta["Project_ID"].isin(inner_train_studies)) & mask
        inner_train_ds = prepare_dataset(Subset(corpus, np.where(inner_train_mask)[0]), le)
        
        inner_val_subset = get_study_subset(val_study, None)
        inner_val_ds = prepare_dataset(inner_val_subset, le)
        
        # 2.2 初始化模型 (每次都要重置)
        model = MiCoGPTForSequenceClassification.from_pretrained(
            args.base_model, 
            num_labels=len(classes),
            ignore_mismatched_sizes=True
        )
        
        # 2.3 训练配置
        # 注意：内层不需要 load_best_model_at_end，我们需要完整的 loss 曲线
        inner_output_dir = f"{args.output_root}/temp_inner/{test_study}/{val_study}"
        training_args = TrainingArguments(
            output_dir=inner_output_dir,
            num_train_epochs=MAX_EPOCHS,
            per_device_train_batch_size=32,
            per_device_eval_batch_size=32,
            learning_rate=1e-5,
            weight_decay=0.01,
            evaluation_strategy="epoch", # 每个 epoch 测一次
            save_strategy="no",          # 内层为了省空间和时间，不保存 checkpoint
            load_best_model_at_end=False,
            logging_steps=10,
            dataloader_num_workers=0,
            report_to="none"             # 避免 wandb 等干扰
        )
        
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=inner_train_ds,
            eval_dataset=inner_val_ds,
            data_collator=collator,
            compute_metrics=compute_metrics
        )
        
        trainer.train()
        
        # 2.4 提取验证 Loss 曲线
        # log_history 中包含训练 loss 和 eval loss，我们需要筛选出 eval_loss
        history = trainer.state.log_history
        # 提取包含 'eval_loss' 的记录，并按 epoch 排序
        eval_logs = [log for log in history if 'eval_loss' in log]
        # 确保按 epoch 顺序（虽然通常已经是顺序的）
        eval_logs.sort(key=lambda x: x['epoch'])
        
        fold_losses = [log['eval_loss'] for log in eval_logs]
        
        # 简单校验：如果因为某些原因记录数不够，补齐或截断（通常不会发生）
        if len(fold_losses) < MAX_EPOCHS:
            print(f"    Warning: Expected {MAX_EPOCHS} logs, got {len(fold_losses)}. Padding with infinity.")
            fold_losses += [float('inf')] * (MAX_EPOCHS - len(fold_losses))
        
        all_folds_val_losses.append(fold_losses)
        
        # 清理资源
        del model, trainer, inner_train_ds, inner_val_ds
        clean_memory()
    
    # === 3. 聚合结果，选择最佳 Epoch ===
    # 将 list 转为 numpy array: shape (5, MAX_EPOCHS)
    loss_matrix = np.array(all_folds_val_losses)
    # 计算每个 epoch 的平均 loss
    avg_loss_per_epoch = np.mean(loss_matrix, axis=0)
    
    # 找到 loss 最小的索引 (epoch 从 1 开始，所以 index + 1)
    best_epoch_idx = np.argmin(avg_loss_per_epoch)
    best_epoch = int(best_epoch_idx + 1)
    min_loss = avg_loss_per_epoch[best_epoch_idx]
    
    print(f"\n[Result] Best Epoch for {test_study} is: {best_epoch} (Avg Val Loss: {min_loss:.4f})")
    
    # === 4. Refit: 用最佳 Epoch 在整个训练池上重训 ===
    print(f"[Refit] Retraining on all 5 training studies for {best_epoch} epochs...")
    
    # 4.1 准备 Refit 数据 (所有 5 个训练 study 合并)
    refit_train_mask = (meta["Project_ID"].isin(train_pool_studies)) & mask
    refit_train_ds = prepare_dataset(Subset(corpus, np.where(refit_train_mask)[0]), le)
    
    # 准备最终测试数据
    final_test_subset = get_study_subset(test_study, None)
    final_test_ds = prepare_dataset(final_test_subset, le)
    
    print(f"  Refit Train Samples: {len(refit_train_ds)}")
    print(f"  Final Test Samples:  {len(final_test_ds)}")
    
    # 4.2 初始化模型
    model = MiCoGPTForSequenceClassification.from_pretrained(
        args.base_model, 
        num_labels=len(classes),
        ignore_mismatched_sizes=True
    )
    
    # 4.3 训练配置
    final_output_dir = f"{args.output_root}/finetuned/{test_study}"
    training_args = TrainingArguments(
        output_dir=final_output_dir,
        num_train_epochs=best_epoch,     # <--- 关键：使用选出的最佳 Epoch
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        learning_rate=1e-5,
        weight_decay=0.01,
        evaluation_strategy="no",        # Refit 阶段不需要验证
        save_strategy="no",              # Refit 结束后直接保存或预测，不需要中间存 checkpoint
        logging_steps=10,
        dataloader_num_workers=0
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=refit_train_ds,
        data_collator=collator
    )
    
    trainer.train()
    
    # === 5. 最终预测与保存 ===
    print(f"[Predict] Predicting on {test_study}...")
    predictions = trainer.predict(final_test_ds)
    
    eval_and_save(
        predictions.predictions, 
        predictions.label_ids, 
        [str(c) for c in classes], 
        final_output_dir, 
        activation="softmax"
    )
    
    print(f"Saved results to {final_output_dir}")
    
    # 记录最佳参数信息到文件（可选）
    with open(f"{final_output_dir}/best_params.txt", "w") as f:
        f.write(f"Best Epoch: {best_epoch}\n")
        f.write(f"Avg Val Loss: {min_loss:.4f}\n")
        f.write(f"Loss Curve: {avg_loss_per_epoch.tolist()}\n")
    
    # 清理资源
    del model, trainer, refit_train_ds, final_test_ds
    clean_memory()

print("\n=== All LOO Rounds Completed ===")

=== Starting Nested LOO (Double LOO) Fine-tuning ===

>>> Outer Loop: Test Target = PRJNA439311

[Inner Loop] Starting 5-fold CV on train pool to find optimal epochs...
  Inner Fold 1/5: Val Study = PRJNA282010


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.641000,2.137470,0.486486,0.318428
2,0.866900,1.297762,0.540541,0.428256
3,0.776100,1.211043,0.540541,0.428256
4,0.699900,1.181512,0.540541,0.428256
5,0.760800,1.118640,0.540541,0.428256
6,0.671600,1.101095,0.540541,0.428256
7,0.645500,1.064079,0.540541,0.428256
8,0.606200,1.022766,0.567568,0.476847
9,0.617400,1.033632,0.540541,0.428256
10,0.637000,1.071269,0.513514,0.375676


  Inner Fold 2/5: Val Study = PRJNA296567


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.578400,0.867381,0.529412,0.522517
2,0.966400,0.828511,0.558824,0.560774
3,0.834900,0.804454,0.558824,0.556034
4,0.732300,0.790668,0.500000,0.487442
5,0.719100,0.770431,0.558824,0.556034
6,0.685600,0.760082,0.529412,0.522517
7,0.639200,0.752105,0.558824,0.556034
8,0.609900,0.747114,0.558824,0.547743
9,0.638300,0.740887,0.588235,0.582202
10,0.586500,0.734767,0.588235,0.582202


  Inner Fold 3/5: Val Study = PRJNA632472


...

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.624800,1.479062,0.585526,0.559420
2,0.856700,1.649219,0.559211,0.508091
3,0.671500,1.617985,0.559211,0.508091
4,0.705300,1.520065,0.565789,0.517692
5,0.636500,1.485298,0.572368,0.522774
6,0.576500,1.431425,0.572368,0.522774
7,0.555700,1.402211,0.572368,0.522774
8,0.636000,1.380247,0.572368,0.522774
9,0.573200,1.348730,0.572368,0.522774
10,0.549300,1.311482,0.572368,0.522774



[Result] Best Epoch for PRJNA820972 is: 30 (Avg Val Loss: 0.9181)
[Refit] Retraining on all 5 training studies for 30 epochs...
  Refit Train Samples: 870
  Final Test Samples:  159


Step,Training Loss
10,1.892000
20,1.570600
30,1.146700
40,1.030900
50,0.894500
60,0.817600
70,0.802100
80,0.740700
90,0.713800
100,0.682900


[Predict] Predicting on PRJNA820972...


Evaluating biome source: False
       TN  FP   FN   TP     Acc      Sn   Sp     TPR  FPR      Rc      Pr  \
t                                                                           
0.000   0  54    0  105  0.6604  1.0000  0.0  1.0000  1.0  1.0000  0.6604   
0.001   0  54    0  105  0.6604  1.0000  0.0  1.0000  1.0  1.0000  0.6604   
0.002   0  54    0  105  0.6604  1.0000  0.0  1.0000  1.0  1.0000  0.6604   
0.003   0  54    0  105  0.6604  1.0000  0.0  1.0000  1.0  1.0000  0.6604   
0.004   0  54    0  105  0.6604  1.0000  0.0  1.0000  1.0  1.0000  0.6604   
...    ..  ..  ...  ...     ...     ...  ...     ...  ...     ...     ...   
0.997  54   0  102    3  0.3585  0.0286  1.0  0.0286  0.0  0.0286  1.0000   
0.998  54   0  103    2  0.3522  0.0190  1.0  0.0190  0.0  0.0190  1.0000   
0.999  54   0  105    0  0.3396  0.0000  1.0  0.0000  0.0  0.0000  0.0000   
1.000  54   0  105    0  0.3396  0.0000  1.0  0.0000  0.0  0.0000  0.0000   
1.001  54   0  105    0  0.3396  0.0000  1.0 